In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
amlcomp_clus = "DW_Cluster"
try:
    amlcomp = ComputeTarget(workspace=ws, name=amlcomp_clus)
except ComputeTargetException:
    comp_cfg = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    
    amlcomp = ComputeTarget.create(ws, amlcomp_clus, comp_cfg)

amlcomp.wait_for_completion(show_output=True)


In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
from azureml.core.environment import Environment

# Specify parameter sampler
ps = RandomParameterSampling({
        "keep_probability": uniform(0.05, 0.1),
        'learning_rate': uniform(0.0005, 0.005),
        'momentum': uniform(0.9, 0.99)
        }
)

# Specify a Policy
bandpolicy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

#This may be irrelevant
#https://docs.microsoft.com/en-us/azure/machine-learning/how-to-train-scikit-learn?view=azure-ml-py
#https://docs.microsoft.com/en-us/azure/machine-learning/how-to-migrate-from-estimators-to-scriptrunconfig?view=azure-ml-py
#TODO Figure out the source directory and arguments requirements
# Create a SKLearn estimator for use with train.py
#est = ### YOUR CODE HERE ###
myenv = Environment(workspace=ws, name="AzureML-Tutorial")
src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      arguments=['--num_epochs', 30, '--output_dir', './training'],
                      compute_target=amlcomp,
                      environment=myenv)

#https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                             hyperparameter_sampling=param_sampling,
                             policy=bandpolicy,
                             primary_metric_name="accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=20, ##Changed from 100
                             max_concurrent_runs=4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hdrive_run = exp.submit(hd_config)
RunDetails(hdrive_run).show()

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
hdrive_run.wait_for_completion(show_output=True)
best_run = hdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n learning rate:',parameter_values[3])
print('\n keep probability:',parameter_values[5])
print('\n batch size:',parameter_values[7])
print('\n FileNames:', best_run.get_file_names())

filename="BestHdriveModel"
#joblib.dump(value=best_run, filename=filename)

#This next line from here may be needed to actually store the model, but try without first
#https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/ml-frameworks/keras/train-hyperparameter-tune-deploy-with-keras/train-hyperparameter-tune-deploy-with-keras.ipynb
#model = best_run.register_model(model_name='keras-mlp-mnist', model_path='outputs/model')
os.makedirs(f'./{filename}', exist_ok=True)
for f in best_run.get_file_names():
    if f.startswith('outputs/model'):
        output_file_path = os.path.join(f'./{filename}', f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        run.download_file(name=f, output_file_path=output_file_path)
        #joblib.dump(value=f, filename=output_file_path)

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(data, separator=',', header=True, encoding='utf8')

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.


automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=3)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
run = experiment.submit(config=automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
filename="BestAutoModel.pkl"
best_run, best_model = local_run.get_output()

os.makedirs(f'./{filename}', exist_ok=True)
for f in best_run.get_file_names():
    if f.startswith('outputs/model'):
        output_file_path = os.path.join(f'./{filename}', f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        run.download_file(name=f, output_file_path=output_file_path)
#         joblib.dump(value=f, filename=output_file_path)